In [ ]:
from VSPEC import ObservationModel
from VSPEC.analysis import PhaseAnalyzer, GCMdecoder
from VSPEC.helpers import to_float
import matplotlib.pyplot as plt
from astropy import units as u, constants as c
import cartopy.crs as ccrs
from os import system
from PIL import Image
from tqdm.auto import tqdm
import numpy as np

CONFIG_FILENAME = 'analysis.cfg'

In [ ]:
model = ObservationModel(CONFIG_FILENAME)
# model.bin_spectra()
# model.build_planet()
# model.build_spectra()

In [ ]:
data = PhaseAnalyzer(model.dirs['all_model'])

In [ ]:
gcm = GCMdecoder.from_psg(model.params.gcm_path)

In [ ]:
geo = model.get_observation_parameters()
plan = model.get_observation_plan(geo)

In [ ]:
def get_image(i):

    fig = plt.figure(figsize=(13,17))
    gs = fig.add_gridspec(4,6)
    spec = fig.add_subplot(gs[0,0:-2])
    system = fig.add_subplot(gs[0,-2:])
    proj = ccrs.Orthographic(
                central_longitude=to_float(plan['planet_sub_obs_lon'][i],u.deg),
                central_latitude=to_float(plan['planet_sub_obs_lat'][i],u.deg))
    temp = fig.add_subplot(gs[1,0:2],projection=proj, fc="r")
    temp.spines['geo']._linewidth = 0.0
    pressure = fig.add_subplot(gs[1,2:4],projection=proj, fc="r")
    pressure.spines['geo']._linewidth = 0.0
    h2o = fig.add_subplot(gs[1,4:],projection=proj, fc="r")
    h2o.spines['geo']._linewidth = 0.0
    albedo = fig.add_subplot(gs[2,:2],projection=proj, fc="r")
    albedo.spines['geo']._linewidth = 0.0
    water = fig.add_subplot(gs[2,2:4],projection=proj, fc="r")
    water.spines['geo']._linewidth = 0.0
    water_ice = fig.add_subplot(gs[2,4:],projection=proj, fc="r")
    water_ice.spines['geo']._linewidth = 0.0
    reflected = fig.add_subplot(gs[3,:3])
    thermal = fig.add_subplot(gs[3,3:])
    phase_bin = 30*u.deg
    phase_step = np.diff(plan['phase'])[0]
    noise_factor = np.sqrt(phase_step/phase_bin)
    flux = (data.spectrum('thermal',i)+data.spectrum('reflected',i))/data.spectrum('total',i)*1e6
    noise = noise_factor*data.spectrum('noise',i)/data.spectrum('total',i)*1e6
    spec.plot(data.wavelength,to_float(flux,u.dimensionless_unscaled))
    spec.fill_between(data.wavelength.value,
                to_float(flux-noise,u.dimensionless_unscaled),
                to_float(flux+noise,u.dimensionless_unscaled),color='k',alpha=0.2)
    spec.set(xlabel='wavelength (um)',ylabel='Planet Flux (ppm)')
    # spec.set_ylim(-3,95)
    scale_type = 'linear'
    spec.set_xscale(scale_type)
    spec.set_yscale(scale_type)


    geo.get_system_visual(plan['phase'][i],system)

    var = 'Tsurf'
    im = temp.pcolormesh(gcm.get_lons(),gcm.get_lats(),(gcm[var]),
                transform=ccrs.PlateCarree(),
                cmap='viridis',
            )
    fig.colorbar(im,ax=temp,label=var,orientation='horizontal')

    var = 'Psurf'
    im = pressure.pcolormesh(gcm.get_lons(),gcm.get_lats(),(10**gcm[var]),
                transform=ccrs.PlateCarree(),
                cmap='viridis',
            )
    fig.colorbar(im,ax=pressure,label=var,orientation='horizontal')

    R = model.params.planet_radius
    g = model.params.planet_grav*u.m/u.s**2
    M = g*R**2/c.G

    var = 'H2O'
    im = h2o.pcolormesh(gcm.get_lons(),gcm.get_lats(),gcm.get_column_density(var,M,R).cgs,
                transform=ccrs.PlateCarree(),
                cmap='viridis',
            )
    fig.colorbar(im,ax=h2o,label=var,orientation='horizontal')

    var = 'Albedo'
    im = albedo.pcolormesh(gcm.get_lons(),gcm.get_lats(),(gcm[var]),
                transform=ccrs.PlateCarree(),
                cmap='viridis',
            )
    fig.colorbar(im,ax=albedo,label=var,orientation='horizontal')

    var = 'Water'
    im = water.pcolormesh(gcm.get_lons(),gcm.get_lats(),gcm.get_column_clouds(var,M,R),
                transform=ccrs.PlateCarree(),
                cmap='viridis',
            )
    fig.colorbar(im,ax=water,label=var,orientation='horizontal')

    var = 'WaterIce'
    im = water_ice.pcolormesh(gcm.get_lons(),gcm.get_lats(),gcm.get_column_clouds(var,M,R),
                transform=ccrs.PlateCarree(),
                cmap='viridis',
            )
    fig.colorbar(im,ax=water_ice,label=var,orientation='horizontal')

    phase = data.unique_phase
    ref = data.lightcurve('reflected',0)
    therm = data.lightcurve('thermal',140)
    reflected.plot(phase[:i],ref[:i])
    reflected.scatter(phase[i],ref[i])
    reflected.set_xlim(left=np.min(phase.value),right=np.max(phase.value))
    reflected.set_ylim(top=np.max(ref.value)*1.05,bottom=0)
    reflected.set(xlabel='phase',ylabel=f'flux at {data.wavelength[0]:.1f}')

    thermal.plot(phase[:i],therm[:i])
    thermal.scatter(phase[i],therm[i])
    thermal.set_xlim(left=np.min(phase.value),right=np.max(phase.value))
    thermal.set_ylim(top=np.max(therm.value)*1.05,bottom=np.min(therm.value)*0.9)
    thermal.set(xlabel='phase',ylabel=f'flux at {data.wavelength[140]:.1f}')

    fig.subplots_adjust(wspace=0.55)

    return fig

In [ ]:
get_image(1).show()

In [ ]:
total = data.N_images
system('mkdir temp')
for i in tqdm(range(total),desc='Generating images',total=total):
    get_image(i).savefig(f'temp/temp{i}.png',facecolor='w',dpi=120)
    plt.close('all')
frames = []
for i in tqdm(range(total),desc='Building list',total=total):
    fname = f'temp/temp{i}.png'
    frames.append(Image.open(fname))
frame_one = frames[0]
frame_one.save('gcm.gif', format="GIF", append_images=frames, save_all=True, duration=30, loop=True)
system('rm -r temp')

In [ ]:
i=0
fig = plt.figure(figsize=(7,7))
proj = ccrs.Orthographic(
                central_longitude=to_float(plan['planet_sub_obs_lon'][i],u.deg),
                central_latitude=to_float(plan['planet_sub_obs_lat'][i],u.deg))
ax = fig.add_subplot(projection=proj, fc="r")
ax.spines['geo']._linewidth = 0.0
var = 'Tsurf'
R = model.params.planet_radius
g = model.params.planet_grav*u.m/u.s**2
grav_mode = model.params.planet_grav_mode
M = g*R**2/c.G
im = ax.pcolormesh(gcm.get_lons(),gcm.get_lats(),(gcm.get_column_density('H2O',M,R)).cgs,
            transform=ccrs.PlateCarree(),
            cmap='viridis',
        )
fig.colorbar(im,ax=ax,label=var)
lons = np.linspace(0,360,91)
ax.plot(lons,lons*0,c='w',transform=ccrs.PlateCarree(),ls='--')
lats = np.linspace(-90,90,45)
ax.plot(lats*0,lats,c='w',transform=ccrs.PlateCarree(),ls='--')
